## Setup and data load & wrangle

In [22]:
from datetime import datetime, timedelta
import numpy as np  
import pandas as pd
import plotly.graph_objs as go
import seaborn as sns
import enrich_transactions
import os
import glob
from format_transactions import format_transactions
from functools import reduce
import pandas as pd
from typing import List, Union

import plotly.graph_objects as go
import pandas as pd


### Set some parameters

In [23]:
pd.set_option('display.width', 170)  # Adjust the value as needed
monitor = '4k'
# monitor = '1080p'
width_, height_ = (2200, 1100) if monitor == '4k' else (1000, 700)

### Created functions go here

In [24]:
def join_two_dfs(left: pd.DataFrame, right: pd.DataFrame, join_columns: List[str]) -> pd.DataFrame:
    '''
    Helper function to outer-join two DataFrames and merge their overlapping columns.
    '''
    # Get overlapping columns (excluding join columns)
    join_cols_set = set(join_columns)
    overlap_cols = set(left.columns) & set(right.columns) - join_cols_set
    # Merge DataFrames with outer join
    merged = pd.merge(left, right, on=join_columns, how='outer')
    # For each overlapping column, merge them using coalesce
    for col in overlap_cols:
        merged[col] = merged[f'{col}_x'].combine_first(merged[f'{col}_y'])
        merged.drop([f'{col}_x', f'{col}_y'], axis=1, inplace=True)
    return merged

def join_dataframes(dataframes: List[pd.DataFrame], 
                   join_columns: Union[str, List[str]]) -> pd.DataFrame:
    '''
    Perform an inner join across a list of DataFrames, merging columns with the same name.
    dataframes: List of DataFrames to join
    join_columns: Union[str, List[str]] - Column name(s) to join on. Can be a single string or list of strings
        
    Returns: df - Joined DataFrame with merged columns  
    '''
    # Input validation
    if not dataframes:
        raise ValueError('No DataFrames provided')
    if len(dataframes) < 2:
        raise ValueError('Need at least 2 DataFrames to join')
    # Convert join_columns to list if string
    if isinstance(join_columns, str):
        join_columns = [join_columns]
    # Use reduce to join all DataFrames
    result = reduce(lambda left, right: join_two_dfs(left, right, join_columns), 
                   dataframes)
    # Remove duplicates based on join columns, keeping first occurrence
    result = result.drop_duplicates(subset=join_columns)
    return result

def get_full_data(folder_path):
    '''
    Function to find all data CSVs and merge them into one big df
    Returns df
    '''
    # Get all CSV files in the specified folder
    csv_paths = glob.glob(os.path.join(folder_path, '*.csv')) 
    if not csv_paths:
        raise FileNotFoundError('No CSV files found in the specified directory.')
    csv_files = [pd.read_csv(path) for path in csv_paths]
    df_all_time = join_dataframes(csv_files, 'Transaction ID')
    return df_all_time


data_folder = '/Users/peter/Library/CloudStorage/GoogleDrive-petersargentgcse@gmail.com/My Drive/sources/bank_data/monzo'
df_raw = get_full_data(data_folder)

In [26]:
df_raw

,Transaction ID,Type,Local currency,Category,Emoji,Receipt,Local amount,Time,Name,Category split,Money In,Amount,Address,Money Out,Description,Currency,Notes and #tags,Date
0,tx_00009rSwKq5ZYItXdcApof,Faster payment,GBP,Income,NaN,NaN,74.84,16:39:26,Sargent Peter,NaN,74.84,74.84,NaN,NaN,ME,GBP,ME,28/01/2020
1,tx_00009rV1snea0A5O38rrsX,Card payment,GBP,Eating out,☕,NaN,-20.00,16:51:01,Starbucks,NaN,NaN,-20.00,325 Kentish Town Road,-20.00,STARBUCKS LONDON GBR,GBP,NaN,29/01/2020
2,tx_00009rVFdyUGq35N4dUePK,Card payment,GBP,Groceries,🍏,NaN,-16.58,19:25:12,Sainsbury’s,NaN,NaN,-16.58,836 High Road,-16.58,SAINSBURYS SACAT 0513 NORTH FINCHLE GBR,GBP,NaN,29/01/2020
3,tx_00009rYvZSMl6LqB1zndUw,Card payment,GBP,Shopping,🔩,NaN,-1.99,13:59:08,Screwfix,NaN,NaN,-1.99,High Road,-1.99,SCREWFIX NORTH FINCHLE LONDON GBR,GBP,NaN,31/01/2020
4,tx_00009rZPtGMASXnYbQqhvg,Card payment,GBP,Groceries,🍏,NaN,-18.44,19:38:52,Sainsbury’s,NaN,NaN,-18.44,836 High Road,-18.44,SAINSBURYS SACAT 0513 NORTH FINCHLE GBR,GBP,NaN,31/01/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6980,tx_0000AnXJViGQkERzFp6gQk,Card payment,GBP,Transport,🚴,NaN,-3.00,15:39:49,TfL Cycle Hire,NaN,NaN,-3.00,NaN,-3.00,TFL CYCLE HIRE LONDON GBR,GBP,NaN,30/10/2024
6981,tx_0000AnXq6iQqHREiobpBgn,Card payment,GBP,Eating out,🍽,NaN,-7.00,21:45:04,93 Feet East Food,NaN,NaN,-7.00,150 Brick Lane,-7.00,93 Feet East Food London GBR,GBP,NaN,30/10/2024
6982,tx_0000AnY3QIraLv1FzmVhJJ,Card payment,GBP,General,🍸,NaN,-12.40,00:14:17,Old Street Records,NaN,NaN,-12.40,350 -354 Old Street,-12.40,OLD STREET RECORDS London GBR,GBP,NaN,31/10/2024
6983,tx_0000AnY3QNVn4NYYPdLrX7,Pot transfer,GBP,Savings,NaN,NaN,-0.60,00:14:17,Savings Pot,NaN,NaN,-0.60,NaN,-0.60,Round up,GBP,NaN,31/10/2024


In [ ]:
# def get_latest_csv(folder_path):
#     '''
#     Function to find the latest-created csv in the folder
#     Returns csv file path
#     '''
#     # Get all CSV files in the specified folder
#     csv_files = glob.glob(os.path.join(folder_path, '*.csv')) 
#     if not csv_files:
#         raise FileNotFoundError('No CSV files found in the specified directory.')
#     # Get the most recently added file
#     latest_file = max(csv_files, key=os.path.getctime)
#     return latest_file

# def read_latest_csv(folder_path):
#     '''
#     Takes the file path and reads the csv file into a df
#     Returns df of monzo transaction data (normally all-time transactions)
#     '''
#     latest_csv = get_latest_csv(folder_path)
#     # Read the CSV file using pandas
#     df = pd.read_csv(latest_csv)
#     return df

In [16]:
def create_month_col(df_, dt_col='tr_datetime'):
    """
    Creates a column for the month of data in year-month format
    Returns the df with new column
    """
    df_[dt_col] = pd.to_datetime(df_[dt_col])
    # df_['month'] = df_[dt_col].dt.to_period('M')
    df_['month'] = df_[dt_col].dt.strftime('%Y-%m')
    return df_

def resample_datetime(df_, dt_col='tr_datetime'):
    """
    Must be done for waterfall charts
    Takes in a df with many rows per month (per-minute dt data)
    Returns with just one per month, aggregated by sum
    """
    # Set the date column as the index
    df_ = df_.set_index(dt_col)
    # Aggregate data by month
    df_ = df_.resample('M').sum().reset_index()
    return df_


def forward_filler(start_date, n_days_):
    """
    Creates a df of dates filled forwad by :n_days: days.
    Returns date df
    """
    # Generate sequence of dates
    date_range = pd.date_range(start=start_date, periods=n_days_, freq='D')
    # Create new df with date column
    df_ = pd.DataFrame({'date': date_range})
    df_['date'] = date_range
    # We will have salary and other things on last day of month
    df_['is_month_end'] = df_['date'] + pd.offsets.MonthEnd(0) == df_['date']
    return df_


def assign_categories(df_, classifications, based_on='name'):
    """
    Re-assigns the category based on info from another column; default to 'name' col
    Returns the amended transactions df
    """
    if 'category_m' not in df_.columns:
        df_['category_m'] = df_['category']
    for category, bases in classifications.items():
        df_.loc[df_[based_on].isin(bases), 'category_m'] = category
    return df_


### Load data then format and enrich

In [18]:
# I was using the Monzo API to get data. However there is less data available via this method.
# The data is richer if you manually export an all-time bank statement from the app.
# This is saved in the google drive folder below

# data folder location
data_folder = '/Users/peter/Library/CloudStorage/GoogleDrive-petersargentgcse@gmail.com/My Drive/sources/bank_data/monzo'
df_raw = get_full_data(data_folder)
# Call the function and get the dataframe
df_raw = read_latest_csv(data_folder)
df_transactions = format_transactions(df_raw)

### Do some exclusions

In [116]:
# Top all-time transactions
df_exclusions = df_transactions
excl_pattern_regex = 'pot'
excl_list_name = ['vanguard', 'peter sargent', 'sargent peter', 'vanguard asset management limited c']
# this pot transaction accounts for a £230 discrepancy. however not 100% sure that the discrep IS CAUSED BY this transaction
# keep_ids = ['tx_0000AkQ3sbHLmfKhfywazR']

# Keep rows where ((type is NOT pot-related) AND (name is NOT in excl list)) OR (id in keep_ids)
df_exclusions = df_exclusions[((~df_exclusions['type'].str.contains(excl_pattern_regex, regex=True)) & (~df_exclusions['name'].isin(excl_list_name))) | (df_exclusions['transaction_id'].isin(keep_ids))]

### Annotate some for future reference

In [117]:
name_annotations = {
    'adyen' : 'some_transfer_service',
    'doctap same day gp' : 'health check for montagn\'hard',
    'ee' : 'jamf_phone_contract',
    'etoro' : 'dont_remember_buying',
    'mp luz' : 'camera_shop_são_paulo',
    'nash lp/2019' : 'some_meal',
    'vitesse' : 'some_adventure_la_paz',
    'wastecareli eb' : 'unknown',
}

In [118]:
df_transactions = enrich_transactions.main('', from_file=False, df_=df_exclusions)
# Move this as needed
# Any further manipulations should first create new df from the base one
df_transactions_base = df_transactions

[WARN] KeyError: could not drop time. Not found in df columns
[WARN] KeyError: could not drop emoji. Not found in df columns
[WARN] KeyError: could not drop currency. Not found in df columns
[WARN] KeyError: could not drop notes_and_tags. Not found in df columns
[WARN] KeyError: could not drop address. Not found in df columns
[WARN] KeyError: could not drop receipt. Not found in df columns
[WARN] KeyError: could not drop category_split. Not found in df columns


## Create over-time waterfall chart

In [119]:
df_waterfall_monthly = df_transactions_base
df_waterfall_monthly = df_waterfall_monthly[['tr_datetime', 'amount', 'money_out', 'money_in']]

# Prepare data for waterfall chart
df_waterfall_monthly = resample_datetime(df_waterfall_monthly)
df_waterfall_monthly = create_month_col(df_waterfall_monthly)
# Add a starting value
start_row = pd.DataFrame({'tr_datetime': [pd.NaT], 'month': ['start'], 'amount': [0]})
df_waterfall_monthly = pd.concat([start_row, df_waterfall_monthly], ignore_index=True)

### Budget forecast
change this heading name it's a bit off

In [120]:
## Set up basic forecasting data model. Tinker here to try different scenarios
dict_forecast_model = {
    'monthly_budget' : -1000,
    'rent' : -700,
    'salary_projection' : 1796.20,
    'monthly_loan_repayment' : -50,
    'adjustments' : {
        'peel_surplus' : -80,
        'initial_correction' : 128.52
    }
}
# There are an average of 30.437 days in each calendar month
dict_forecast_model['daily_budget'] = dict_forecast_model['monthly_budget']/30.437

In [121]:
## Now complete data model
# Create the other table - days going forwards
balance_0 = df_transactions_base.iloc[-1]['balance']
start_date = df_transactions_base.iloc[-1]['tr_datetime'].date()

# Initialise forecast df with the date range
n_days = 90
df_forecast = forward_filler(start_date, n_days)
# We must make sure that date of final row is the end of month
forecast_eom = df_forecast['is_month_end'].iloc[-1]
if not forecast_eom:
    # We must fill until month end
    end_date_0 = df_forecast['date'].iloc[-1]
    # Find the end of the month
    month_end = end_date_0 + pd.offsets.MonthEnd(0)
    # Calculate the number of days until the end of the month - add fill-forward number
    n_days += (month_end - end_date_0).days
    # Regenerate the df with new n_days
    df_forecast = forward_filler(start_date, n_days)

# Initialise a col to contain the forecast balance change +/- each day
df_forecast['daily_sum'] = dict_forecast_model['daily_budget']
# TEMPORARY, REMOVE IN AUGUST
idx_end_july = df_forecast[df_forecast['date']=='2024-07-31	'].index
df_forecast.loc[idx_end_july, 'is_month_end'] = False
# Now fill daily summation col with logic:
#        if month_end then add salary and loan repayment
#        elif 14th day then add rent
#        else preserve existing sum
df_forecast['daily_sum'] = np.where(
    df_forecast['is_month_end'],
    df_forecast['daily_sum'] + dict_forecast_model['monthly_loan_repayment'] + dict_forecast_model['salary_projection'],
    np.where(
        df_forecast['date'].dt.day == 14,
        df_forecast['daily_sum'] + dict_forecast_model['rent'],
        df_forecast['daily_sum']
    )
)
adjust_balance_0 = balance_0 + dict_forecast_model['daily_budget'] + sum([adjustment for adjustment in dict_forecast_model['adjustments'].values()])
# Now add initial balance. Must be done in separate np.where as it's not mutually exclusive with the logic above (start_date could be 30th/14th)
df_forecast['balance_projected'] = df_forecast['daily_sum'].shift(fill_value=0).cumsum() + adjust_balance_0

In [122]:
## Now bring to same format for waterfall
df_forecast['month'] = df_forecast['date'].dt.strftime('%Y-%m')
df_forecast_pivot = df_forecast.pivot_table(
    values='daily_sum',
    index='month',
    aggfunc='sum'
).sort_values('month').reset_index()
df_forecast_pivot = df_forecast_pivot.rename(columns={'daily_sum': 'amount'})

df_waterfall_monthly = pd.concat([df_waterfall_monthly, df_forecast_pivot], ignore_index=True)

### Add shaded regions and lines to chart
These are useful to show my living circumstances at the time of spend/gain

In [123]:
## Create shaded region coordinates for each region
# Generate a colour palette
palette = sns.color_palette('Set1')
opacity_bg = 0.2
new_palette = []
for colour in palette:
    new_colour = list(colour)
    new_colour.append(opacity_bg)
    new_palette.append(tuple(new_colour))


# Define region dates
regions_start_end = {
    'uni' :
        {'start': datetime(2016, 9, 1), 'end' : datetime(2019, 7, 31)},
    'tap' :
        {'start': datetime(2020, 2, 1), 'end' : datetime(2021, 6, 1)},
    'jamf' :
        {'start': datetime(2021, 6, 2), 'end' : datetime(2023, 2, 4)},
    'south_america' :
        {'start': datetime(2023, 2, 5), 'end' : datetime(2023, 12, 10)},
    'return' :
        {'start': datetime(2023, 12, 11), 'end' : datetime(2024, 3, 1)},
    'yacht' :
        {'start': datetime(2024, 3, 2), 'end' : datetime.now().date()},
    'palladium' :
        {'start': datetime(2024, 6, 17), 'end' : datetime.now().date()},
}
add_shapes = []
palette_index = 1
for name, dates in regions_start_end.items():
    add_shapes.append(
        {'x0': dates['start'],
         'x1': dates['end'],
         'y0': df_waterfall_monthly['amount'].cumsum().min()-1000,
         'y1': df_waterfall_monthly['amount'].cumsum().max()+1000,
         'fillcolor': f'rgba{new_palette[palette_index]}',
         'line': {'width': 0}
        }
    )
    palette_index+=1

In [124]:
## Set up vertical lines for covid lockdown dates
date_0 = '2020-03-23'
date_1 = '2021-07-19'

def vert_line_params():
    return {
        'type' : 'line',
        'y0' : -500,
        'y1' : df_waterfall_monthly['amount'].cumsum().max()+1000,
        'line' : {
            'color' : 'rgba(0, 0, 0, 0.4)',
            'width' : 1,
        }
    }

line_0 = vert_line_params()
line_1 = vert_line_params()
line_0.update({
    'x0' : date_0,
    'x1' : date_0,
})
line_1.update({
    'x0' : date_1,
    'x1' : date_1,
})
add_shapes.extend([line_0, line_1])

In [125]:
## Create the Waterfall Chart
fig_0 = go.Figure(go.Waterfall(
    x=df_waterfall_monthly['month'],
    y=df_waterfall_monthly['amount'],
    textposition='outside',
    text=df_waterfall_monthly['amount'],
    connector={'line':{'color':'rgba(63, 63, 63, 0)'}},
))
# Update layout with various tweaks
fig_0.update_layout(
    title='Monthly Aggregated Waterfall Chart',
    waterfallgap=0.1,
    # showlegend=True,
    # Add shading
    shapes=add_shapes,
    width=width_,
    height=height_,
    yaxis_tickprefix = '£',
    yaxis_tickformat = ',.0f',
    # Default view to last 1y with interactive slider
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1y', step='year', stepmode='backward'),
                dict(count=2, label='2y', step='year', stepmode='backward'),
                dict(step='all')
            ])
            
        ),
        rangeslider=dict(visible=True)
    ),
    # Set default axis limits
    yaxis=dict(
        range=[-100, df_waterfall_monthly['amount'].cumsum().max() + 1000]
    ),
)
fig_0.show()

## Drill down

In [126]:
## Drill down into categories
excl_cats = ['bills', 'income']
df_post_pan_0 = df_transactions_base[
    (df_transactions_base['tr_datetime'] > '2023-11-01')
    # & (df_transactions_base['tr_datetime'] < '2022-12-31')
    & ~df_transactions_base['category'].isin(excl_cats)
    & df_transactions_base['amount'] > 0
]
# Create more-useful categories for top-spending items. Dict will take names and assign categories to those entries
category_classification = {
    'atm' : ['atm'],
    'bills' : ['karim mwebe', 'dfa passport office', 'compare the man & van', 'enfield removals'],
    'city travel' : ['transport for london', 'tfl cycle hire', 'uber'],
    # 'shopping' : ['h&m'],
    'drinking' : ['patch st paul', 'the raven', 'sugar cane island ltd', 'the asparagus', 'old street brewery', 'rising sun',
                  'infernos', 'the swan', 'wetherspoon', 'the experiment', 'the old school yard', 'lost society', 'battersea brewery',
                  'the mason\'s arms', 'prince of wales', 'artillery arms - ec1', 'tremenheere', 'dog & fox'],
    'eating out' : ['flavour of naples', 'windmill clapham', 'canteen', 'mcdonald\'s'],
    'events' : ['dice', 'ep x trailpurs', 'the top secret comedy club', 'let\'s do this', 'electric brixton', 'the cloak and dagger',
                'peckham audio', 'the four thieves', 'looking glass cocktail club', 'jaguar shoes collective', 'eventim uk',
                'todaytix', 'colours hoxton', 'tdr productions lt', 'paris versailles', 'ticketswap', 'rumours', 'hoxton cabin'],
    'groceries' : ['majorelle'],
    'health' : ['specsavers', 'oticaboavista', 'drogaria vidifarma ltd'],
    'holidays' : ['aj', 'booking.com'],
    'friends_fam' : ['christopher sargent', 'susan sargent', 'charles roberts', 'jonathan peel', 'fred varley', 'cristian banks',
                     'luke parker', 'joseph purle', 'ross macdonald', 'thomas gruhl'],
    'online shopping' : ['amazon', 'ebay'],
    'perks' : ['the gym', 'better leisure gym'],
    'personal care' : ['rush-tower bridge'],
    'photo' : ['perfect pictures', 'the range penzance'],
    'sport' : ['londonclimbingcentres', 'substation', 'vauxwall east', 'vauxwall west', 'castle climbing ce'],
    'sport equipment' : ['decathlon', 'go beyond sport', 'nike', 'cotswold outdoor', 'millets', 'charles birch limited',
                         'mountain warehouse', 'sp bbco headwear'],
    'subscriptions' : ['spotify', 'adobe', 'adobe creative cloud', 'google play', 'google'],
}
excl_transactions = {
    'tx_0000AhkOSVgln300aORYUD' : 'marine discovery p',
}
df_post_pan_0 = assign_categories(df_post_pan_0, category_classification)
# df_post_pan_0 = create_month_col(df_post_pan_0)
# resample_datetime(df_post_pan_0)

/var/folders/r9/0bsf6g_s5njccgsljtpzdm3m0000gn/T/ipykernel_29690/3649220282.py:45: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [127]:
df_post_pan_ = df_post_pan_0[['name', 'amount', 'category_m']].groupby(['name', 'category_m']).sum().reset_index()
df_post_pan_ = df_post_pan_.pivot(
    index=['name', 'category_m'],
    columns=[],
    values='amount'
).reset_index().sort_values(0)
df_post_pan_.head(60)

,name,category_m,0
134,tap,holidays,-474.09
8,atm,atm,-416.87
167,yacht inn,entertainment,-379.37
40,ebay,online shopping,-211.38
90,mytrip 1025216549,holidays,-207.99
5,amazon,online shopping,-192.96
152,trainline,transport,-158.67
135,tesco,groceries,-137.07
91,national express,transport,-129.40
78,marine discovery p,holidays,-120.84


In [128]:
create_month_col(df_post_pan_0)

/var/folders/r9/0bsf6g_s5njccgsljtpzdm3m0000gn/T/ipykernel_29690/3649220282.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/r9/0bsf6g_s5njccgsljtpzdm3m0000gn/T/ipykernel_29690/3649220282.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,transaction_id,type,name,category,amount,local_amount,local_currency,description,money_out,money_in,tr_datetime,balance,category_m,month
5725,tx_0000AbLUZoS3tSZaXdr8yn,card payment,vid doces,groceries,-3.71,-22.50,brl,vid doces rio de janeir bra,-3.71,NaN,2023-11-01 00:22:04,997.93,groceries,2023-11
5729,tx_0000AbOpTT9spT3ccLoKFH,card payment,tap,holidays,-474.09,-2811.93,brl,tap madrid bra,-474.09,NaN,2023-11-02 15:00:22,517.84,holidays,2023-11
5732,tx_0000AbPAws3JS139i70hdq,card payment,cat s kiosk lanches e,shopping,-1.65,-10.00,brl,cat s kiosk lanches e rio de janeir bra,-1.65,NaN,2023-11-02 19:00:59,516.19,shopping,2023-11
5733,tx_0000AbPO9cZNcphOifGPZJ,card payment,atm,entertainment,-35.74,-214.00,brl,3019803 io de janeirr bra,-35.74,NaN,2023-11-02 21:28:57,480.45,atm,2023-11
5734,tx_0000AbPOV6Ku9KOpJwEoVe,card payment,drogaria vidifarma ltd,general,-2.08,-12.46,brl,drogaria vidifarma ltd rio de janeir bra,-2.08,NaN,2023-11-02 21:32:50,478.37,health,2023-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6334,tx_0000AkX1Gtfg76b4LFgoFO,card payment,beigel bake,entertainment,-6.90,-6.90,gbp,beigel bake london gbr,-6.90,NaN,2024-08-01 18:39:09,3420.99,entertainment,2024-08
6335,tx_0000AkXZUKU58E0sGOPUHr,card payment,mol wex rental,general,200.00,200.00,gbp,mol*wex rental \fixation\442073835127 \...,NaN,200.0,2024-08-02 01:02:33,3620.99,general,2024-08
6337,tx_0000AkYIUxxqqDoCBkV4E5,card payment,photobook ltd,eating out,-4.00,-4.00,gbp,zettle_*photobook ltd london gbr,-4.00,NaN,2024-08-02 09:26:53,3596.99,eating out,2024-08
6339,tx_0000AkYWKMv37OTHjv06Xy,card payment,beigel shop,groceries,-3.40,-3.40,gbp,beigel shop london e1 6 gbr,-3.40,NaN,2024-08-02 12:01:51,3633.59,groceries,2024-08


In [129]:
## Bar chart for top-spending categories
# Function to identify top 5 categories and create 'Other'
def process_month(df_month):
    # Calculate the total amount by category for each month
    category_totals = df_month.groupby('category_m')['amount'].sum()
    # Identify top 5 categories
    top_categories = category_totals.nlargest(20).index
    # Replace non-top categories with 'Other'
    df_month['category_m'] = df_month['category_m'].apply(lambda x: x if x in top_categories else 'other')
    # Aggregate amounts
    df_processed = df_month.groupby(['month', 'category_m'])['amount'].sum().reset_index()
    return df_processed


## Create bar chart for per-month category breakdown
df_bar_post_pan = df_post_pan_0
df_bar_post_pan = df_bar_post_pan[['amount', 'category_m', 'month']]

# Reformat and pivot for bar chart
df_bar_post_pan = df_bar_post_pan.groupby(['month', 'category_m']).sum().reset_index()
df_bar_post_pan = df_bar_post_pan[df_bar_post_pan['amount'] < 0]
df_bar_post_pan['amount'] *= -1
df_bar_post_pan = df_bar_post_pan.groupby('month').apply(process_month).reset_index(drop=True)


# Calculate the total number of rows
total_rows = len(df_bar_post_pan)

# Count the occurrences of each unique value in the 'category' column
category_counts = df_bar_post_pan['category_m'].value_counts()

# Calculate the percentage contribution of each category
category_percentage = (category_counts / total_rows) * 100

# Map the calculated percentages back to the DataFrame
df_bar_post_pan['category_percentage'] = df_bar_post_pan['category_m'].map(category_percentage)
df_bar_post_pan = df_bar_post_pan.sort_values(['category_percentage', 'category_m'], ascending=False)


# Pivot for bar chart
df_bar_pivot = df_bar_post_pan.pivot(
    index='month',
    columns='category_m',
    values='amount'
).reset_index()
# Add a total column
# df_bar_pivot['total'] = df_bar_pivot.iloc[:,2:].sum(axis=1)
# bar_series = list(df_bar_post_pan['category_m'].unique())
# bar_series.extend(['total'])

# Create the bar chart
fig_3 = go.Figure()
# for category in df_bar_post_pan['category_m'].unique():
#     fig_3.add_trace(go.Bar(
#         x=df_bar_pivot['month'],
#         y=df_bar_pivot[category],
#         name=category
#     ))
# Add bars for each category
for category in df_bar_post_pan['category_m'].unique():
    # df_category = df_bar_pivot[df_bar_pivot['category_m'] == category]
    fig_3.add_trace(go.Bar(
        x=df_bar_pivot['month'],
        y=df_bar_pivot[category],
        name=category
    ))

fig_3.update_layout(
    barmode='stack',
    title='Spending categories',
    yaxis_title='Amount',
    width=width_,
    height=height_,    
)
# Show the plot
fig_3.show()

TODO: next 2 things I really need:
    - things together on a dashboard with shared filtering
    - itemised table to show the month in question

### Older analysis

In [130]:
## [de-activated] Line chart
colour_danger_zone = 'rgba(232, 49, 60, 0.2)'
shape_danger_zone = {
    'x0': start_date + timedelta(days=-1),
    'x1': df_forecast['date'].iloc[-1].date() + timedelta(days=1),
    'y0': 500,
    'y1': -200,
    'fillcolor': colour_danger_zone,
    'line': {'width': 2, 'color': colour_danger_zone}
}

# Create the line chart
fig_1 = go.Figure()

fig_1.add_trace(go.Scatter(
    x=df_forecast['date'],
    y=df_forecast['balance_projected'],
    mode='lines',
    name='Values',
    line=dict(color='royalblue'),
))

# Update layout
fig_1.update_layout(
    title='Projected Balance over Time',
    xaxis_title='Date',
    yaxis_title='Projected Balance',
    shapes=[shape_danger_zone],
    width=width_,
    height=height_,
    # Set default axis limits
    yaxis=dict(
        range=[0, df_forecast['balance_projected'].max() + 100]
    ),
    xaxis=dict(
        range=[start_date, df_forecast['date'].iloc[-1].date()]
    )
)

# Show the plot
fig_1.show()


In [131]:
# How much have I spent in the past month?
today = datetime.now()
df_1_mo = df_transactions[df_transactions['tr_datetime'] >= today-timedelta(days=30)]
df_1_mo

,transaction_id,type,name,category,amount,local_amount,local_currency,description,money_out,money_in,tr_datetime,balance
6237,tx_0000Ajb5MGW3jcOYvnVB9l,card payment,elif food centre,general,-3.98,-3.98,gbp,elif food centre london gbr,-3.98,NaN,2024-07-04 19:51:41,1025.23
6238,tx_0000Ajb6I2p2o2izZ8OYin,card payment,tfl cycle hire,transport,-1.65,-1.65,gbp,tfl cycle hire london gbr,-1.65,NaN,2024-07-04 20:02:08,1023.58
6239,tx_0000Ajbk1zQxdII3tAYDgn,card payment,transport for london,transport,-1.75,-1.75,gbp,tfl travel charge tfl.gov.uk/cp gbr,-1.75,NaN,2024-07-05 03:27:26,1021.83
6240,tx_0000Ajcx5rADocjUpHM83d,card payment,toogoodt ef2486n632370,eating out,-5.00,-5.00,gbp,toogoodt ef2486n632370 toogoodtogo.c dnk,-5.00,NaN,2024-07-05 17:28:30,1016.83
6241,tx_0000AjeNu09pRXyIxLh8Jl,card payment,professional langu,entertainment,-5.00,-5.00,gbp,sq *professional langu london gbr,-5.00,NaN,2024-07-06 10:03:36,1011.83
...,...,...,...,...,...,...,...,...,...,...,...,...
6336,tx_0000AkXfYTLwhPIxZgcPJp,direct debit,id mobile,bills,-20.00,-20.00,gbp,10903125/001,-20.00,NaN,2024-08-02 02:10:32,3600.99
6337,tx_0000AkYIUxxqqDoCBkV4E5,card payment,photobook ltd,eating out,-4.00,-4.00,gbp,zettle_*photobook ltd london gbr,-4.00,NaN,2024-08-02 09:26:53,3596.99
6338,tx_0000AkYJHiVh5X8AzwBpXF,faster payment,beatriz de alvarenga nogueira,income,40.00,40.00,gbp,ml 20240731-000,NaN,40.0,2024-08-02 09:35:42,3636.99
6339,tx_0000AkYWKMv37OTHjv06Xy,card payment,beigel shop,groceries,-3.40,-3.40,gbp,beigel shop london e1 6 gbr,-3.40,NaN,2024-08-02 12:01:51,3633.59


In [132]:
# Anomalous transactions (should exclude for prediction)
df_1_mo = df_1_mo.sort_values('amount', ascending=True).set_index('transaction_id')
df_1_mo.head(10)

,type,name,category,amount,local_amount,local_currency,description,money_out,money_in,tr_datetime,balance
transaction_id,,,,,,,,,,,
tx_0000Ak0De7ZFOuSFRLC2i6,faster payment,hoxton house ltd,bills,-700.00,-700.00,gbp,rent jul-aug,-700.00,NaN,2024-07-16 22:52:20,1023.00
tx_0000AkQ3upG2mWLqoIj6CB,card payment,mol wex rental,general,-234.32,-234.32,gbp,mol*wex rental 442073835127 gbr,-234.32,NaN,2024-07-29 10:05:29,651.37
tx_0000AkA4EP7WYqiFLL25y6,card payment,mytrip 1025216549,holidays,-207.99,-207.99,gbp,mytrip_1025216549 online gbr,-207.99,NaN,2024-07-21 16:53:57,988.57
tx_0000AkAHqJkyFZqcg5U90z,card payment,ebay,shopping,-164.69,-164.69,gbp,ebay o*01-11855-11839 london gbr,-164.69,NaN,2024-07-21 19:26:28,823.88
tx_0000AjgXavk71MAZpQ8iZO,faster payment,m a kaci,bills,-29.78,-29.78,gbp,council tax,-29.78,NaN,2024-07-07 11:01:38,952.57
tx_0000AkBZpE3ZwwUphLLwCf,card payment,resident advisor,entertainment,-27.50,-27.50,gbp,resident advisor ticke london gbr,-27.50,NaN,2024-07-22 10:22:41,796.38
tx_0000AjhrFvjO3t4osPHxcQ,direct debit,community fibre,bills,-24.69,-24.69,gbp,a685512,-24.69,NaN,2024-07-08 02:16:39,929.65
tx_0000AjjRVvNGwkYzVXEZQg,card payment,tonkotsu clapham,eating out,-23.51,-23.51,gbp,tonkotsu clapham london gbr,-23.51,NaN,2024-07-08 20:37:38,905.74
tx_0000AjgddnsvIBDRQK54Fu,card payment,aldi,groceries,-22.93,-22.93,gbp,aldi (781-112) london gbr,-22.93,NaN,2024-07-07 12:09:23,921.54


In [ ]:
df_1_mo.tail(10)

In [ ]:
df_1_mo = df_1_mo.drop(['tx_0000AiuhpP1IddgljCOu49', # deposit
'tx_0000AjBOIfFmfMYVKsvBvl', # transfer to PJPS Starling
'tx_0000Aiv8WC9AyOv4fEIoV7', # cornwall train
'tx_0000AjKWk8VgXz5pU15Sk5']) # parking permits

In [ ]:
df_1_mo['money_out'].sum() - 700
# -£1410.81

In [ ]:
# df_1_mo['count_col'] = 1
df_1_mo_pivot = pd.pivot_table(
    df_1_mo,
    index='description',          # Column to group by
    values='money_out',        # Column to aggregate
    aggfunc={'money_out': ['sum', 'count']}  # Aggregation functions
)
df_1_mo_pivot.sort_values('sum')

# conclusion: no more transactions that need excluding. £1410 is an accurate spending prediction
# conclusion: grocery spend £270 (7 vendors). Could likely be a little less by sticking to aldi


In [ ]:
df_transactions[df_transactions['description']=='shop and go old street london gbr']

In [ ]:
df_1_mo[df_1_mo['description'].str.contains('tonkotsu.')]

In [ ]:
df_1_mo = df_1_mo.sort_values('tr_datetime')
for i, row in df_1_mo.iterrows():
    print('\t'.join([str(row['amount']), row['description'], str(row['tr_datetime'])]))
